In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config, sub_root_path
from bcnf.train import Trainer
from bcnf import CondRealNVP_v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'trajectory_TRF_small'

In [3]:
config_path_pattern = os.path.join("{{BCNF_ROOT}}", "configs", "runs", f"{MODEL_NAME}.yaml")
config_path = sub_root_path(config_path_pattern)
print(f'Loading config from {config_path}')

config = load_config(config_path, verify=False)

Loading config from /home/psaegert/Projects/bcnf/configs/runs/trajectory_TRF_small.yaml


In [4]:
model = CondRealNVP_v2.from_config(config).to(device)

print(f'{model.n_params:,}')

128,129


In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 4/4 [01:17<00:00, 19.45s/it, file=fixed_data_render_2s_15FPS_4.pkl]


Using trajectories data for training. Shapes:
Conditions: [torch.Size([4000, 30, 3])]
Parameters: torch.Size([4000, 19])


In [6]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -41.6789 - Val: -45.3278 (avg: -45.6078, min: -45.6524) | lr: 3.91e-07 - Patience: 500/500 - z: (0.0230 ± 0.1540) ± (0.7490 ± 0.1923):  19%|█▉        | 9555/50000 [1:48:32<7:39:25,  1.47it/s]  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


distance_to_last_best_val_loss_fold_-1,▁▁▁▁▁▁▁▁▂▂▁▁▁▄▂▁▁▃▃▂▃▃▁▃▁▄▄▃▃▁▁▁▃▄▃▂▁▆▄█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr_fold_-1,███████████████████▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
time_fold_-1,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
train_loss_fold_-1,█▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_fold_-1,█▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
z_mean_mean_fold_-1,▄▆▆▇█▅▂▂▄▅▁▂▆▆▁▅▆▄▅▅▃▄▅▃▂▄▇█▄▆▇▅▅▅▄▄▅▄▅▇
z_mean_std_fold_-1,▁▂▂▂▃▄▃▅▃▄█▅▄▃▆▄█▅▅▄▃▃▂▄▄▃▃▄▃▃▂▂▂▂▂▁▂▂▁▁
z_std_mean_fold_-1,█▇▆▅▅▄▅▅▂▃▅▃▄▅▃▅▂▅▃▆▃▂▄▃▄▃▃▄▂▄▂▄▂▁▃▁▂▃▁▄
z_std_std_fold_-1,▂▇▇▄▅▃██▅▇▆▄▅▅▂▆▃▂▂▆▁▁▆▂▅▃▂▂▃▅▃▇▆▃▆▂▂▆▄▆
distance_to_last_best_val_loss_fold_-1,500


In [7]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path_pattern}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_TRF_small
